In [ ]:
!wget https://storage.googleapis.com/chrome-for-testing-public/132.0.6834.159/win64/chrome-win64.zip
!unzip chrome-win64.zip
!pip install selenium bs4

In [ ]:
from bs4 import BeautifulSoup
try:
    from selenium import webdriver

    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    import pandas as pd
    import bolas

except:
    print("Configurando ambiente")
    import os
    os.system("pip install selenium wget pandas")
    import wget
    import zipfile
    
    if "chromedriver" not in os.listdir():
        print("Downloading chromedriver")
        filename = wget.download("https://storage.googleapis.com/chrome-for-testing-public/132.0.6834.159/win64/chrome-win64.zip")
        with zipfile.ZipFile(f"./{filename}", 'r') as zip_ref:
            zip_ref.extractall("./chromedriver")
    else:
        print("Chromedrive found")
    
    from selenium import webdriver
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    import pandas as pd

options = webdriver.ChromeOptions()
options.binary_location = "./chromedriver/chrome-win64/chrome.exe"
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-web-security')
options.add_argument('--disable-site-isolation-trials')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument('--disable-notifications')
options.page_load_strategy = 'eager'

driver = webdriver.Chrome(options=options)

def articleFormatter(article, tag): 
    return {
        "title": article.find("h3").text.strip(),
        "link": article.find("a").get_attribute_list("href")[0],
        "data": article.find("div", class_="date").text.strip(),
        "tag": tag
    }


def SCfilter(article):
    cidades_sc1 = pd.read_excel('./planilhas/cidade_sc1.xlsx')
    
    for key in [" sc ", "santa catarina", " sc", "sc "]:
        if key in article["title"].lower():
            return True
        
        
    for key in ["-sc-", "santa-catarina", "-sc", "sc-"]:
        if key in article["link"].split("/")[-1].lower():
            return True

    for cidade in cidades_sc1["MUNICIPIO"]:
        if cidade.lower() in article["title"].lower() or cidade.lower() in article["link"].split("/")[-1].lower():
            return True

    return False

def getNewsByTags(tags):
    allNews = []

    pageCounter = 0
    for tag in tags.keys():
        for page in range(tags[tag]):
            if pageCounter % 10 == 0:
                print(f"\nPlanilha salva com {len(allNews)} notícias para backup...")
                storeAsExcel(allNews)
                print("Salvo\n")
                
            print(f'Buscando notícias com a tag: {tag}')

            driver.get(f"https://www.nsctotal.com.br/tag/{tag}?page={page}")
            try:
                WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, "date") ) )
                driver.implicitly_wait(5)
            except:
                continue
            
            print("Página acessada com sucesso")
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            news = soup.find_all('div', class_='featured-news-thumb')
            
            parsedNews = [articleFormatter(article, tag) for article in news]

            parsedNews = list(filter(SCfilter, parsedNews))
            
            allNews += parsedNews
            print(f"Coletadas mais {len(parsedNews)} notícias, total {len(allNews)}")

            pageCounter += 1
        
        
            
    return allNews

def storeAsExcel(data):
    rows = list(map(lambda article: article.values(), data))
    df = pd.DataFrame(rows, columns=["title", "link", "data", "tag"])
    df.drop_duplicates()
    df.to_excel("./planilhas/noticias.xlsx", index=False)
    

searchReference = {
    "chuvas": 52,
    "chuva em sc": 35,
    "chuvas em sc": 60,
    "chuva": 395,
    "chuva forte": 5,
    "chuvarada": 11,
    "temporal": 109,
    "tempestade": 21,
    "ciclone": 32,
    "ciclone bomba": 5,
    "ciclone extratropical": 6,
    "previsao do tempo": 708,
    "frente fria": 9,
    "enchente": 92,
    "enchentes": 22,
    "alagamento": 61,
    "alagamentos": 20,
    "deslizamento": 36,
    "deslizamentos": 9,
    "deslizamento de terra": 5
}

searchReference = {tag.replace(" ", "-"): searchReference[tag] for tag in searchReference}

print(searchReference)

data = getNewsByTags(searchReference)

print(data)

storeAsExcel(data)

Configurando ambiente
Chromedrive found
{'chuvas': 52, 'chuva-em-sc': 35, 'chuvas-em-sc': 60, 'chuva': 395, 'chuva-forte': 5, 'chuvarada': 11, 'temporal': 109, 'tempestade': 21, 'ciclone': 32, 'ciclone-bomba': 5, 'ciclone-extratropical': 6, 'previsao-do-tempo': 708, 'frente-fria': 9, 'enchente': 92, 'enchentes': 22, 'alagamento': 61, 'alagamentos': 20, 'deslizamento': 36, 'deslizamentos': 9, 'deslizamento-de-terra': 5}

Planilha salva com 0 notícias para backup...
Salvo

Buscando notícias com a tag: chuvas
Página acessada com sucesso
Coletadas mais 9 notícias, total 9
Buscando notícias com a tag: chuvas
Página acessada com sucesso
Coletadas mais 9 notícias, total 18
Buscando notícias com a tag: chuvas
Página acessada com sucesso
Coletadas mais 9 notícias, total 27
Buscando notícias com a tag: chuvas
Página acessada com sucesso
Coletadas mais 3 notícias, total 30
Buscando notícias com a tag: chuvas
Página acessada com sucesso
Coletadas mais 5 notícias, total 35
Buscando notícias com a t

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=132.0.6834.159)
Stacktrace:
	GetHandleVerifier [0x00007FF7AB6F02F5+28725]
	(No symbol) [0x00007FF7AB652AE0]
	(No symbol) [0x00007FF7AB4E510A]
	(No symbol) [0x00007FF7AB4D1C2C]
	(No symbol) [0x00007FF7AB4D1AF0]
	(No symbol) [0x00007FF7AB4E7B21]
	(No symbol) [0x00007FF7AB57FA7A]
	(No symbol) [0x00007FF7AB55FD53]
	(No symbol) [0x00007FF7AB52A0E3]
	(No symbol) [0x00007FF7AB52B471]
	GetHandleVerifier [0x00007FF7ABA1F30D+3366989]
	GetHandleVerifier [0x00007FF7ABA312F0+3440688]
	GetHandleVerifier [0x00007FF7ABA278FD+3401277]
	GetHandleVerifier [0x00007FF7AB7BAAAB+858091]
	(No symbol) [0x00007FF7AB65E74F]
	(No symbol) [0x00007FF7AB65A304]
	(No symbol) [0x00007FF7AB65A49D]
	(No symbol) [0x00007FF7AB648B69]
	BaseThreadInitThunk [0x00007FFB6AA67374+20]
	RtlUserThreadStart [0x00007FFB6B45CC91+33]
